In [1]:
import os
import cv2
import numpy as np
import tqdm as tqdm
import torch.optim as optim
import matplotlib.pyplot as plt

PREPARE_IMAGES = True
IMAGES_TO_PREPARE = 100


C:\Users\luche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CatsAndDogs:
    def __init__(self, rootFolderWithSubFolders):
        foldersCount = len(os.listdir(rootFolderWithSubFolders))
        if foldersCount != 2:
            raise Exception("Root folder containd more than 2 folders")

        
        self.IMAGE_SIZE = 50
        CATS = "../data/pets/cats/"
        DOGS = "../data/pets/dogs/"
        LABELS = {CATS: 0, DOGS: 1}

    training_data = []
    cat_counter = 0
    dog_counter = 0

    def prepare_training_dataset(self):
        for dirs in self.LABELS:
            for f in os.listdir(dirs):
                try:
                    path = os.path.join(dirs, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMAGE_SIZE, self.IMAGE_SIZE))
                    self.training_data.append(
                        [np.array(img), np.eye(2)[self.LABELS[dirs]]])
                except Exception as e:
                    pass
        np.random.shuffle(self.training_data)
        np.save("training_data.[]", self.training_data)


In [3]:
obj = CatsAndDogs()
obj.prepare_training_dataset()


C:\Users\luche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\lib\npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
myDataset = CatsAndDogs("")

In [11]:
for dir in obj.LABELS:
    print(dir)

../data/pets/cats/
../data/pets/dogs/


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_1 = nn.Conv2d(1, 32, 5)
        self.conv_layer_2 = nn.Conv2d(32, 64, 5)
        self.conv_layer_3 = nn.Conv2d(64, 128, 5)

        test_data = torch.randn(50, 50).view(-1, 1, 50, 50)
        self.convert_to_linear = None

        self.run_conv_layers(test_data)

        self.hidden_layer_1 = nn.Linear(self.convert_to_linear, 512)
        self.output_layer = nn.Linear(512, 2)

    def run_conv_layers(self, data):
        data = F.max_pool2d(F.relu(self.conv_layer_1(data)), (2, 2))
        data = F.max_pool2d(F.relu(self.conv_layer_2(data)), (2, 2))
        data = F.max_pool2d(F.relu(self.conv_layer_3(data)), (2, 2))

        if self.convert_to_linear is None:
            self.convert_to_linear = data[0].shape[0] * \
                data[0].shape[1] * data[0].shape[2]
        return data

    def forward(self, data):
        data = self.run_conv_layers(data)
        data = data.view(-1, self.convert_to_linear)
        data = F.relu(self.hidden_layer_1(data))
        data = self.output_layer(data)

        return F.softmax(data, dim=1)


In [6]:
my_net = ConvNet()


In [8]:
optimizer = optim.Adam(my_net.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
X = torch.Tensor([i[0] for i in obj.training_data]).view(-1, 1, 50, 50)
X = X/255.0
Y = torch.Tensor([i[1] for i in obj.training_data])

VALIDATION_PERCENTAGE = 0.1

validation_size = int(len(X) * VALIDATION_PERCENTAGE)

train_data = X[:-validation_size]
correct_answers = Y[:-validation_size]

test_data = X[-validation_size:]
test_correct_answers = Y[-validation_size:]

BATCH_SIZE = 128
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in range(0, len(train_data), BATCH_SIZE):
        batch_X = train_data[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_Y = correct_answers[i:i+BATCH_SIZE]
        my_net.zero_grad()
        results = my_net(batch_X)
        loss = loss_fn(results, batch_Y)
        loss.backward()
        optimizer.step()
print(loss)


tensor(0.1702, grad_fn=<MseLossBackward0>)


In [9]:
correct = 0
total = 0

with torch.no_grad():
    for i in range(len(test_data)):
        correct_answer = torch.argmax(test_correct_answers[i])
        result = my_net(test_data[i].view(-1, 1, 50, 50))[0]
        model_answer = torch.argmax(result)

        if correct_answer == model_answer:
            correct += 1
        total += 1
print("Correct:", correct)
print("Total: ", total)
print(correct/total*100)


Correct: 1861
Total:  2494
74.61908580593423
